In [1]:
import pandas as pd
import numpy as np
import pymatgen as mg
import json
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from pymatgen.analysis.local_env import JMolNN
from pymatgen.core.periodic_table import Element
from itertools import combinations
%matplotlib inline

In [3]:
list_elements = ['Al', 'Ga', 'In', 'O']
df_elements = pd.DataFrame([Element(sym).data for sym in ['Al', 'Ga', 'In', 'O']], index=list_elements)
use_cols = ['Atomic mass', 'Atomic radius', 'Boiling point', 'Common oxidation states', 'Ionic radii', 'Liquid range', 'Melting point', 'Molar volume', 'Thermal conductivity', 'Van der waals radius', 'Velocity of sound', 'X', ]
df_elements[use_cols]


,Atomic mass,Atomic radius,Boiling point,Common oxidation states,Ionic radii,Liquid range,Melting point,Molar volume,Thermal conductivity,Van der waals radius,Velocity of sound,X
Al,26.981539,1.25,2792 K,[3],{'3': 0.675},1858.53 K,933.47 K,10.00 cm<sup>3</sup>,235 W m<sup>-1</sup> K<sup>-1</sup>,1.84,5100 m s<sup>-1</sup>,1.61
Ga,69.723000,1.30,2477 K,[3],{'3': 0.76},2174.09 K,302.91 K,11.80 cm<sup>3</sup>,29 W m<sup>-1</sup> K<sup>-1</sup>,1.87,2740 m s<sup>-1</sup>,1.81
In,114.818000,1.55,2345 K,[3],{'3': 0.94},1915.25 K,429.75 K,15.76 cm<sup>3</sup>,82 W m<sup>-1</sup> K<sup>-1</sup>,1.93,1215 m s<sup>-1</sup>,1.78
O,15.999400,0.60,90.2 K,[-2],{'-2': 1.26},35.4 K,54.8 K,17.36 cm<sup>3</sup>,0.02658 W m<sup>-1</sup> K<sup>-1</sup>,1.52,317.5 m s<sup>-1</sup>,3.44


In [5]:
pd.options.display.max_columns = 100
df_sk = pd.read_csv('data/elemental_data.csv', index_col=0)
df_sk.loc[list_elements, :]

,bp,cp_g,cp_mol,e_fusion,e_vapor,ea,group,ie1,ie2,kai-a,kai-p,mass,mol_vol,mp,period,ratom,rcov,rho,rps-d,rps-p,rps-s,rvdw,thermal_cond,z
Al,2792.0,0.897,24.200,10.700,290.8000,42.5,13.0,577.5,1816.7,1.613,1.61,26.9815,0.000010,933.47,3.0,118.0,1.24,2.700,NaN,0.905,0.770,1.84,2.370000,13.0
Ga,2477.0,0.373,26.030,5.590,256.0600,28.9,13.0,578.8,1979.3,1.756,1.81,69.7230,0.000012,302.91,4.0,136.0,1.23,5.904,0.17,0.935,0.760,1.87,0.406000,31.0
In,2345.0,0.233,26.740,3.260,226.3500,28.9,13.0,558.3,1820.7,1.656,1.78,114.8180,0.000016,429.75,5.0,156.0,1.42,7.310,0.36,1.110,0.940,1.93,0.816000,49.0
O,90.2,0.918,29.378,0.222,3.4109,141.0,16.0,1313.9,3388.3,3.610,3.44,15.9994,0.011196,54.80,2.0,48.0,0.64,1.429,NaN,0.180,0.285,1.52,0.000267,8.0


In [10]:
df = df_elements.loc[:, use_cols]

kJ_to_eV = 96.48455
df['Molar specific heat'] = df_sk['cp_mol']  # J / (mol \cdot K)
df['Heat of fusion'] = df_sk['e_fusion']  / kJ_to_eV  # eV
df['Heat of vapor'] = df_sk['e_vapor']  / kJ_to_eV  # eV
df['Electron affinity'] = df_sk['ea']  / kJ_to_eV  # eV
df['First Ionization energy'] = df_sk['ie1'] / kJ_to_eV  # eV
df['Second Ionization energy'] = df_sk['ie2'] / kJ_to_eV  # eV
df['Covalent radius'] = df_sk['rcov']  # ang
df['Pesudo s orbital radius'] = df_sk['rps-s']  # ang
df['Pesudo p orbital radius'] = df_sk['rps-p']  # ang
df
# density,
# HOMO, LUMO,

,Atomic mass,Atomic radius,Boiling point,Common oxidation states,Ionic radii,Liquid range,Melting point,Molar volume,Thermal conductivity,Van der waals radius,Velocity of sound,X,Molar specific heat,Heat of fusion,Heat of vapor,Electron affinity,First Ionization energy,Second Ionization energy,Covalent radius,Pesudo s orbital radius,Pesudo p orbital radius
Al,26.981539,1.25,2792 K,[3],{'3': 0.675},1858.53 K,933.47 K,10.00 cm<sup>3</sup>,235 W m<sup>-1</sup> K<sup>-1</sup>,1.84,5100 m s<sup>-1</sup>,1.61,24.200,0.110899,3.013954,0.440485,5.985414,18.828921,1.24,0.770,0.905
Ga,69.723000,1.30,2477 K,[3],{'3': 0.76},2174.09 K,302.91 K,11.80 cm<sup>3</sup>,29 W m<sup>-1</sup> K<sup>-1</sup>,1.87,2740 m s<sup>-1</sup>,1.81,26.030,0.057937,2.653896,0.299530,5.998888,20.514165,1.23,0.760,0.935
In,114.818000,1.55,2345 K,[3],{'3': 0.94},1915.25 K,429.75 K,15.76 cm<sup>3</sup>,82 W m<sup>-1</sup> K<sup>-1</sup>,1.93,1215 m s<sup>-1</sup>,1.78,26.740,0.033788,2.345971,0.299530,5.786419,18.870379,1.42,0.940,1.110
O,15.999400,0.60,90.2 K,[-2],{'-2': 1.26},35.4 K,54.8 K,17.36 cm<sup>3</sup>,0.02658 W m<sup>-1</sup> K<sup>-1</sup>,1.52,317.5 m s<sup>-1</sup>,3.44,29.378,0.002301,0.035352,1.461374,13.617724,35.117540,0.64,0.285,0.180


In [36]:
[e for dct in df['Ionic radii'] for _, e in dct.items()]

[0.675, 0.76, 0.94, 1.26]

In [18]:
df.shape

(4, 21)

In [37]:
df['Boiling point'] = df['Boiling point'].str.strip('K').astype
df['Common oxidation states'] = [e[0] for e in df['Common oxidation states']]
df['Ionic radii'] = [e for dct in df['Ionic radii'] for _, e in dct.items()]
df['Liquid range'] = df['Liquid range'].str.strip('K')
df['Melting point'] = df['Melting point'].str.strip('K')
df['Molar volume'] = df['Molar volume'].str.strip('cm<sup>3</sup>')
df['Thermal conductivity'] = df['Thermal conductivity'].str.strip('W m<sup>-1</sup> K<sup>-1</sup>')
df['Velocity of sound'] = df['Velocity of sound'].str.strip('m s<sup>-1</sup>')
df

,Atomic mass,Atomic radius,Boiling point,Common oxidation states,Ionic radii,Liquid range,Melting point,Molar volume,Thermal conductivity,Van der waals radius,Velocity of sound,X,Molar specific heat,Heat of fusion,Heat of vapor,Electron affinity,First Ionization energy,Second Ionization energy,Covalent radius,Pesudo s orbital radius,Pesudo p orbital radius
Al,26.981539,1.25,2792,3,0.675,1858.53,933.47,10.00,235,1.84,5100,1.61,24.200,0.110899,3.013954,0.440485,5.985414,18.828921,1.24,0.770,0.905
Ga,69.723000,1.30,2477,3,0.760,2174.09,302.91,11.80,29,1.87,2740,1.81,26.030,0.057937,2.653896,0.299530,5.998888,20.514165,1.23,0.760,0.935
In,114.818000,1.55,2345,3,0.940,1915.25,429.75,15.76,82,1.93,215,1.78,26.740,0.033788,2.345971,0.299530,5.786419,18.870379,1.42,0.940,1.110
O,15.999400,0.60,90.2,-2,1.260,35.4,54.8,17.36,0.02658,1.52,317.5,3.44,29.378,0.002301,0.035352,1.461374,13.617724,35.117540,0.64,0.285,0.180


In [43]:
df = df.astype(float)

In [44]:
joblib.dump(df, 'data/elements_property.joblib')

['data/elements_property.joblib']

In [45]:
joblib.load('data/elements_property.joblib')

,Atomic mass,Atomic radius,Boiling point,Common oxidation states,Ionic radii,Liquid range,Melting point,Molar volume,Thermal conductivity,Van der waals radius,Velocity of sound,X,Molar specific heat,Heat of fusion,Heat of vapor,Electron affinity,First Ionization energy,Second Ionization energy,Covalent radius,Pesudo s orbital radius,Pesudo p orbital radius
Al,26.981539,1.25,2792.0,3.0,0.675,1858.53,933.47,10.00,235.00000,1.84,5100.0,1.61,24.200,0.110899,3.013954,0.440485,5.985414,18.828921,1.24,0.770,0.905
Ga,69.723000,1.30,2477.0,3.0,0.760,2174.09,302.91,11.80,29.00000,1.87,2740.0,1.81,26.030,0.057937,2.653896,0.299530,5.998888,20.514165,1.23,0.760,0.935
In,114.818000,1.55,2345.0,3.0,0.940,1915.25,429.75,15.76,82.00000,1.93,215.0,1.78,26.740,0.033788,2.345971,0.299530,5.786419,18.870379,1.42,0.940,1.110
O,15.999400,0.60,90.2,-2.0,1.260,35.40,54.80,17.36,0.02658,1.52,317.5,3.44,29.378,0.002301,0.035352,1.461374,13.617724,35.117540,0.64,0.285,0.180
